#  EPA-122A *Spatial* Data Science


## Lab 5 - Part 1: Plotting, Simple Linear Regression, K-NN Regression

**TU Delft**<br>
**Q2 2024**<br>
**Instructors:** Giacomo Marangoni, Theodoros Chatzivasileiadis <br>

---

# Table of Contents

* [Learning Goals](#learning)
* [Review of  the  `numpy` Python library](#review)
    * [Reshaping an array](#reshaping)
    * [Array Slicing (a reminder...)](#aslicing)
* [Pandas Slicing (a reminder...)](#pslicing)
    * [Positioning with `.iloc` and `loc`](#iloc)
    * [Pandas slicing. a short example](#slicingex)
* [Towards matplotlib and beyond (a reminder...)](#matplotlib)
    * [Refreshing some `matplotlib` skills ](#skills)
    * [Figures with subplots](#subplots)
    * [Additional matplotlib resources](#additional)
* [Simple Linear Regression](#linear)
    * [Linear Regression. a first practical example](#linearex)
        * [Linear Regression: Formulae](#formulae)
    * [Building our first linear regression model](#firstlinear)
        * [Exercise 1: Function definition](#ex1)
        * [Exercise 2: Visualising results](#ex2)
* [Building a model with `statsmodels` and `sklearn`](#stats-sk)
    * [`statsmodels` and `sklearn`: an operational difference](#opdifference)
    * [Using the `statsmodels` package](#stats)
    * [Using the `sklearn` package](#sk)
        * [Additional information on the `scikit-learn` library](#addinfo)
    * [Practice with `sklearn`: Obtaining training and testing sets from a dataframe](#sk-practice)
* [$k$ Nearest Neighbors](#kNN)

In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("display.notebook_repr_html", True)
import warnings

warnings.filterwarnings("ignore")
# Displays the plots for us.
%matplotlib inline

<a class="anchor" id="learning"></a>
# Learning Goals

By the end of this lab, you should be able to:
* Review `numpy` including 2-D arrays and understand array reshaping
* Use `matplotlib` to make plots
* Feel comfortable with simple linear regression
* Feel comfortable with $k$ nearest neighbors

<a class="anchor" id="review"></a>
# Review of  the  `numpy` Python library

Throughout the first lab (Lab 00) we learned about the `numpy` library [(documentation)](http://www.numpy.org/) and its fast array structure, called the `numpy array`.
These two concepts (numpy library and arrays) will play a crucial role when we want to explore regression models and techniques. Thus, let us jump right back into them and introduce some new ideas along the way!

In [ ]:
# We can create an array
my_array = np.array([1, 4, 9, 16])
my_array

We can now try to identify the 'shape' of our array.

In [ ]:
print(f"Size of my array: {my_array.size}, or length of my array: {len(my_array)}")
print(f"Shape of my array: {my_array.shape}")

One important thing to notice is how the array shape is displayed:

* For a 1D array, .shape returns a tuple with 1 element (n,)
* For a 2D array, .shape returns a tuple with 2 elements (n,m)
* For a 3D array, .shape returns a tuple with 3 elements (n,m,p)

<a class="anchor" id="reshaping"></a>
## Reshaping an array

As we will see later on in this lab, array reshaping can be a pretty useful tool when dealing with regression models and techniques.
We are particularly interested in understanding how to reshape a 1D array to a 2D array.

In [ ]:
# How to reshape a 1D array to a 2D
my_array.reshape(2, 2)

Numpy arrays support the same operations as lists! Below we slice and iterate.

In [ ]:
print("array[2:4]:", my_array[2:4])  # A slice of the array

# Iterate over the array
for ele in my_array:
    print("element:", ele)

Also, when working with arrays it is importnat to remember that `numpy` gains a lot of its efficiency from being **strongly typed** (all elements are of the same type, such as integer or floating point). If the elements of an array are of a different type, `numpy` will force them into the same type (the longest in terms of bytes)

In [ ]:
mixed = np.array([1, 2.3, "eleni", True])
print(type(1), type(2.3), type("eleni"), type(True))
mixed  # all elements will become strings

<a class="anchor" id="aslicing"></a>
## Array Slicing (a reminder...)

Numpy arrays can be sliced, and can be iterated over with loops.  Below is a schematic illustrating slicing two-dimensional arrays

 ![header](figs/2dindex_v2.png)

Notice that the list slicing syntax still works!

`array[2:,3]` says "in the array, get rows 2 through the end, column 3"
`array[3,:]` says "in the array, get row 3, all columns".

<a class="anchor" id="pslicing"></a>
# Pandas Slicing (a reminder...)

As we will understand later, slicing can become really useful when dealing with regression models. But, considering that a big chunk of the data we'll analyse throughout the course is collected in `pandas dataframe` structures, refreshing some `pandas` slicing concepts will help us in the next steps.

But, before we can look into pandas slicing techniques it can useful to look again into the difference between `.iloc` and `.loc`.

<a class="anchor" id="iloc"></a>
## Positioning with `.iloc` and `loc`

Look at the following data frame. It is a bad example because we have duplicate values for the index but that is legal in pandas. It's just a bad practice and we are doing it to illustrate the difference between positioning with `.iloc` and `loc`. To keep rows unique, though, internally, `pandas` has its own index which in this dataframe runs from `0` to `2`.

In [ ]:
index = ["A", "Z", "A"]
famous = pd.DataFrame(
    {
        "Elton": ["singer", "Candle in the wind", "male"],
        "Maraie": ["actress", "Do not know", "female"],
        "num": np.random.randn(3),
    },
    index=index,
)
famous

In [ ]:
# accessing elements by label can bring up duplicates!!
famous.loc["A"]  # since we want all rows is the same as famous.loc['A',:]

In [ ]:
# accessing elements by position is unique - brings up only one row
famous.iloc[1]

To summarise, `.iloc` is "position-based" (position is unique) while `.loc` is "label-based" (label is not unique).

<a class="anchor" id="slicingex"></a>
## Pandas Slicing: a short example

In [ ]:
# import cast dataframe
cast = pd.read_csv("data/cast.csv", encoding="utf_8")
cast.head()

In [ ]:
# briefly analysing the shape of our dataframe
cast.shape

In [ ]:
# getting rows 10 to 13 (python slicing style : exclusive of end)
cast.iloc[10:13]

In [ ]:
# get me columns 0 to 2 but all rows - use head()
cast.iloc[:, 0:2].head()

In [ ]:
# get me rows 10 to 13 AND only columns 0 to 2
cast.iloc[10:13, 0:2]

In [ ]:
# COMPARE: get me rows 10 to 13 (pandas slicing style : inclusive of end)
cast.loc[10:13]

In [ ]:
# give me columns 'year' and 'type' by label but only for rows 5 to 10
cast.loc[5:10, ["year", "type"]]

<a class="anchor" id="matplotlib"></a>
# Towards matplotlib and beyond (a reminder...)
<br>
<img style="float: center" src="https://imgs.xkcd.com/comics/convincing.png">

As we already explorede in the previous labs `matplotlib` is a very powerful `python` library for making scientific plots.

We will not focus too much on the internal aspects of `matplotlib` in today's lab. There are many excellent tutorials out there for `matplotlib`.  For example,
* [`matplotlib` homepage](https://matplotlib.org/)
* [`matplotlib` tutorial](https://github.com/matplotlib/AnatomyOfMatplotlib)

Conveying your findings convincingly is an absolutely crucial part of any analysis. Therefore, you must be able to write well and make compelling visuals.  Creating informative visuals is an involved process and we won't cover that in this lab.  However, part of creating informative data visualizations means generating *readable* figures.  If people can't read your figures or have a difficult time interpreting them, they won't understand the results of your work.  Here are some non-negotiable commandments for any plot:
* Label $x$ and $y$ axes
* Axes labels should be informative
* Axes labels should be large enough to read
* Make tick labels large enough
* Include a legend if necessary
* Include a title if necessary
* Use appropriate line widths
* Use different line styles for different lines on the plot
* Use different markers for different lines

There are other important elements, but that list should get you started on your way.

<a class="anchor" id="skills"></a>
## Refreshing some `matplotlib` skills

To refresh our matplolib skills, we can start by generating data we'll use for plotting purposes.

We will use the following three functions (*don't worry about the math of these functions - they are examples*) to generate new data and create different plots:

* Logistic function:
  \begin{align*}
    f\left(z\right) = \dfrac{1}{1 + be^{-az}}
  \end{align*}
  where $a$ and $b$ are parameters.

* Hyperbolic tangent:
  \begin{align*}
    g\left(z\right) = b\tanh\left(az\right) + c
  \end{align*}
  where $a$, $b$, and $c$ are parameters.

* Rectified Linear Unit:
  \begin{align*}
    h\left(z\right) =
    \left\{
      \begin{array}{lr}
        z, \quad z > 0 \\
        \epsilon z, \quad z\leq 0
      \end{array}
    \right.
  \end{align*}
  where $\epsilon < 0$ is a small, positive parameter.

You are given the code for the first two functions.  Notice that $z$ is passed in as a `numpy` array and that the functions are returned as `numpy` arrays.  Parameters are passed in as floats.

You should write a function to compute the rectified linear unit.  The input should be a `numpy` array for $z$ and a positive float for $\epsilon$.

In [ ]:
def logistic(z: np.ndarray, a: float, b: float) -> np.ndarray:
    """Compute logistic function
    Inputs:
       a: exponential parameter
       b: exponential prefactor
       z: numpy array; domain
    Outputs:
       f: numpy array of floats, logistic function
    """

    den = 1.0 + b * np.exp(-a * z)
    return 1.0 / den


def stretch_tanh(z: np.ndarray, a: float, b: float, c: float) -> np.ndarray:
    """Compute stretched hyperbolic tangent
    Inputs:
       a: horizontal stretch parameter (a>1 implies a horizontal squish)
       b: vertical stretch parameter
       c: vertical shift parameter
       z: numpy array; domain
    Outputs:
       g: numpy array of floats, stretched tanh
    """
    return b * np.tanh(a * z) + c


def relu(z: np.ndarray, eps: float = 0.01) -> np.ndarray:
    """Compute rectificed linear unit
    Inputs:
       eps: small positive parameter
       z: numpy array; domain
    Outputs:
       h: numpy array; relu
    """
    return np.fmax(z, eps * z)

Now let's make some plots.  First, let's just warm up and plot the logistic function.

In [ ]:
x = np.linspace(-5.0, 5.0, 100)  # Equally spaced grid of 100 pts between -5 and 5

f = logistic(x, 1.0, 1.0)  # Generate data

We can now create a first plot using the "standard method".

In [ ]:
plt.plot(x, f)
plt.xlabel("x")
plt.ylabel("f")
plt.title("Logistic Function")
plt.grid(True)

<a class="anchor" id="subplots"></a>
## Figures with subplots

Let's start thinking about the plots as objects. We have the `figure` object which is like a matrix of smaller plots named `axes`. You can use array notation when handling it.

In [ ]:
fig, ax = plt.subplots(1, 1)  # Get figure and axes objects

ax.plot(x, f)  # Make a plot

# Create some labels
ax.set_xlabel("x")
ax.set_ylabel("f")
ax.set_title("Logistic Function")

# Grid
ax.grid(True)

Wow, it's *exactly* the same plot!  Notice, however, the use of `ax.set_xlabel()` instead of `plt.xlabel()`.  The difference is tiny, but you should be aware of it.  We will use this plotting syntax from now on.

What else do we need to do to make this figure better?  Folowing our **plots commandments** we can think of the following options:
* Make labels bigger
* Make title bigger
* Make line thicker
* Make tick mark labels bigger
* Make the grid less pronounced
* Make figure bigger
* Change $x$ boundaries ($x$ goes from $-5$ to $5$)

Let's get to it.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))  # Make figure bigger

# Make line plot
ax.plot(x, f, lw=4)  # making the plot line thicker by specifying its width: lw=...

# Update ticklabel size
ax.tick_params(labelsize=24)

# Creating labels
ax.set_xlabel(
    r"$x$", fontsize=24
)  # making the x-label bigger ; Use TeX for mathematical rendering
ax.set_ylabel(
    r"$f(x)$", fontsize=24
)  # making the y-label bigger; Use TeX for mathematical rendering

# Creating title
ax.set_title("Logistic Function", fontsize=24)  # making the title bigger

# Creating grid
ax.grid(
    True, lw=1.5, ls="--", alpha=0.75
)  # Modifying grid width: lw=... ; Modifying line style: ls=... ; Modifying grid transparency: alpha=...

# Changing x boundaries
ax.set_xlim(-5, 5, emit=False)

Notice:
* `lw` stands for `linewidth`.  We could also write `ax.plot(x, f, linewidth=4)`
* `ls` stands for `linestyle`.
* `alpha` stands for transparency.

<a class="anchor" id="additional"></a>
## Additional `matplotlib` resources

If you want to see all the styles available, please take a look at the documentation.
* [Line styles](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D.set_linestyle)
* [Marker styles](https://matplotlib.org/stable/api/markers_api.html)
* [Everything you could ever want](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D.set_marker)

We haven't discussed it yet, but you can also put a legend on a figure. Here are some additional resources:
* [Legend](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.legend.html)
* [Grid](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.grid.html)

<a class="anchor" id="linear"></a>
# Simple Linear Regression

Linear regression and its many extensions are a workhorse of the statistics and data science community, both in application and as a reference point for other models. Most of the major concepts in machine learning can be and often are discussed in terms of various linear regression models. Thus, this section will introduce you to building and fitting linear regression models and some of the process behind it, so that you can:
1) Fit models to data you encounter;
2) Experiment with different kinds of linear regression and observe their effects;
3) See some of the technology that makes regression models work.

<a class="anchor" id="linearex"></a>
## Linear Regression: a first practical example

We first examine a *toy problem*, focusing our efforts on fitting a linear model to a small dataset with three **observations**.  Each observation consists of one **predictor** $x_i$ and one **response**
$y_i$ for $i = 1, 2, 3$,

\begin{align*}
(x , y) = \{(x_1, y_1), (x_2, y_2), (x_3, y_3)\}.
\end{align*}

To be very concrete, let's set the values of the predictors and responses.

\begin{equation*}
(x , y) = \{(1, 2), (2, 2), (3, 4)\}
\end{equation*}

There is no line of the form $\beta_0 + \beta_1 x = y$ that passes through all three observations, since the data are not collinear. Thus our aim is to find the line that best fits these observations in the *least-squares sense*, as discussed during lecture.

To find the values of $\beta_0$ and $\beta_1$ we can take a closer look to the *fundamental* linear regression formulae presented below.

In [ ]:
# defining our predictor and response arrays (x_train and y_train)
x_train = np.array([1, 2, 3])
y_train = np.array([2, 2, 4])

<a class="anchor" id="formulae"></a>
### Linear Regression: Formulae

Linear regression is special among the models we study because it can be solved explicitly. While most other models (and even some advanced versions of linear regression) must be solved itteratively, linear regression has a formula where you can simply plug in the data.

For the single predictor case it is:
    \begin{align}
      \beta_1 &= \frac{\sum_{i=1}^n{(x_i-\bar{x})(y_i-\bar{y})}}{\sum_{i=1}^n{(x_i-\bar{x})^2}}\\
      \beta_0 &= \bar{y} - \beta_1\bar{x}\
    \end{align}

Where $\bar{y}$ and $\bar{x}$ are the mean of the y values and the mean of the x values, respectively.

With this formulae in mind, let us create our first linear regression model!

<a class="anchor" id="firstlinear"></a>
##  Building our first linear regression model

In this part, we will solve the equations for a simple linear regression and find the best fit solution to our *toy problem*.

The snippets of code below implement the linear regression equations on the given predictors and responses, which we'll call the training data set.

The code below will walk us through the following main actions:

1. Reshaping ou arrays (x_train and y_train) to proper 2D dimensions.
2. Obtaining the mean values of x_train and y_train.
3. Calculating the numerator and denominator of the linear regression formulae.
4. Calculating the values of  $\beta_0$ and $\beta_1$

**NOTE:** We'll understand later why our arrays need to be reshaped into a certain 2D format.

In [ ]:
# Reshaping our arrays to be proper 2D arrays

x_train = x_train.reshape(x_train.shape[0], 1)
y_train = y_train.reshape(y_train.shape[0], 1)

print(x_train.shape, y_train.shape)

In [ ]:
# Computing the mean values of x_train and y_train
y_bar = np.mean(y_train)
x_bar = np.mean(x_train)

In [ ]:
# Calculating numerator and denominator of linear regression formulae
numerator = np.sum((x_train - x_bar) * (y_train - y_bar))
denominator = np.sum((x_train - x_bar) ** 2)

print(numerator.shape, denominator.shape)  # check shapes

Why the empty brackets? The numerator and denominator are scalars, as expected.

In [ ]:
# calculating beta1
beta_1 = numerator / denominator

# calculating beta0
beta_0 = y_bar - beta_1 * x_bar

print("The best-fit line is {0:3.2f} + {1:3.2f} * x".format(beta_0, beta_1))

<a class="anchor" id="ex1"></a>
### Exercise 1: Function definition

Turn the code from the above cells into a function called `simple_linear_regression_fit`, that inputs the training data and returns `beta0` and `beta1`.

To do this, copy and paste the code from the above cells below and adjust the code as needed, so that the training data becomes the input and the betas become the output.

```python
def simple_linear_regression_fit(x_train: np.ndarray, y_train: np.ndarray) -> np.ndarray:

    return
```

Check your function by calling it with the training data from above and printing out the beta values.

Your code here
First try it yourself and if it dosesnt work, try again. Still doesn't work? Try working with a friend in groups. Nothing? Ok, use the code below.

In [ ]:
# %load solutions/simple_linear_regression_fit.py

Let's now run this function and see the coefficients.

In [ ]:
"""
betas = simple_linear_regression_fit(x_train, y_train)

beta_0 = betas[0]
beta_1 = betas[1]

print("The best-fit line is {0:8.6f} + {1:8.6f} * x".format(beta_0, beta_1))
"""

<a class="anchor" id="ex2"></a>
### Exercise 2: Visualising results

1. Do the values of `beta0` and `beta1` seem reasonable?
2. Plot the training data using a scatter plot.
3. Plot the best fit line with `beta0` and `beta1` together with the training data.

Your code here
First try it yourself and if it doesn't work, try again. Stil doesn't work? Try working with a friend in groups. Nothing? Ok, use the code below.

In [ ]:
# %load solutions/best_fit_scatterplot.py

The values of `beta0` and `beta1` seem roughly reasonable.  They capture the positive correlation.  The line does appear to be trying to get as close as possible to all the points.

<a class="anchor" id="stats-sk"></a>
# Building a regression model with `statsmodels` and `sklearn`

Now that we can concretely fit the training data from scratch, let's learn two `python` packages to do it all for us:
* [statsmodels](https://www.statsmodels.org/stable/regression.html) and
* [scikit-learn (sklearn)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

Our goal  is to show how to implement simple linear regression with these packages.  For an important sanity check, we compare the $\beta$ values from `statsmodels` and `sklearn` to the $\beta$ values that we found from above with our own implementation.

For the purposes of this lab, `statsmodels` and `sklearn` do the same thing.  More generally though, `statsmodels` tends to be easier for inference \[finding the values of the slope and intercept and dicussing uncertainty in those values\], whereas `sklearn` has machine-learning algorithms and is better for prediction \[guessing y values for a given x value\]. (Note that both packages make the same guesses, it's just a question of which activity they provide more support for.

**Note:** `statsmodels` and `sklearn` are different packages! Unless we specify otherwise, you can use either one.

<a class="anchor" id="opdifference"></a>
## `statsmodels` and `sklearn`: an operational difference

Let's say we a data set of two obsevations with one predictor and one response variable each.

\begin{align*}
(x , y) = \{(x_1, y_1), (x_2, y_2)\}.
\end{align*}

We would then have the following two equations if we run a simple linear regression model.

$$y_1=\beta_0 + \beta_1*x_1$$ $$y_2=\beta_0 + \beta_1*x_2$$ <BR>

For simplicity and calculation efficiency we want to "absorb" the constant $\beta_0$ into an array with $\beta_1$ so we have only multiplication. To do this we introduce the **constant**: ${x}^0=1$<br>

$$y_1=\beta_0*{x_1}^0 + \beta_1*x_1$$ $$y_2=\beta_0 * {x_2}^0 + \beta_1*x_2$$ <BR>

That becomes:

$$y_1=\beta_0*1 + \beta_1*x_1$$ $$y_2=\beta_0 * 1 + \beta_1*x_2$$<bR>

In matrix notation:

$$
\left [
\begin{array}{c}
y_1 \\ y_2 \\
\end{array}
\right] =
\left [
\begin{array}{cc}
1& x_1 \\ 1 & x_2 \\
\end{array}
\right]
\cdot
\left [
\begin{array}{c}
\beta_0 \\ \beta_1 \\
\end{array}
\right]
$$
<BR><BR>

Keeping this in mind, the **operational difference** we wanted to highlight is that while `sklearn` adds the constant for us, `statsmodels` needs us to explicitly add it by using `sm.add_constant`.

**NOTE: Reshaping arrays**

Now that we explored the matrix notation of our linear regression problem we can finally understand why we stressed multiple times the importance of reshaping our predictor and response arrays!

<a class="anchor" id="stats"></a>
## Using the `statsmodels` package

In [ ]:
import statsmodels.api as sm

Below is the code for `statsmodels`.  As we already described above, `statsmodels` does not by default include the column of ones in the $X$ matrix, so we include it manually with `sm.add_constant`.

In [ ]:
# create the X matrix by appending a column of ones (constant) to x_train
X_train = sm.add_constant(x_train)

# let's see the structure of our X matrix
print(X_train)

# build the OLS model (ordinary least squares) from the training data
toyregr_sm = sm.OLS(y_train, X_train)

# do the fit and save regression info (parameters, etc) in results_sm
results_sm = toyregr_sm.fit()

# pull the beta parameters out from results_sm
beta0_sm = results_sm.params[0]
beta1_sm = results_sm.params[1]

# print(f'The regression coefficients from statsmodels are: beta_0 = {beta0_sm} and beta_1 = {beta1_sm}')
print(
    f"The regression coefficients from statsmodels are: beta_0 = {beta0_sm} and beta_1 = {beta1_sm}"
)

Besides the beta parameters, `results_sm` contains a ton of other potentially useful information.

In [ ]:
import warnings

warnings.filterwarnings("ignore")
print(results_sm.summary())

Now let's turn our attention to the `sklearn` library.

<a class="anchor" id="sk"></a>
## Using the `sklearn` package

In [ ]:
from sklearn import linear_model

In [ ]:
# build the least squares model
toyregr = linear_model.LinearRegression()

# save regression info (parameters, etc) in results_skl
results_skl = toyregr.fit(x_train, y_train)

# pull the beta parameters out from results
beta0_skl = toyregr.intercept_[0]
beta1_skl = toyregr.coef_[0][0]


print(
    f"The regression coefficients from the sklearn package are: beta_0 = {beta0_skl} and beta_1 = {beta1_skl}".format(
        beta0_skl, beta1_skl
    )
)

We should feel pretty good about ourselves now, and we're ready to move on to a real problem!

---

<a class="anchor" id="addinfo"></a>
### Additional information on the `scikit-learn` library

Before diving into a "real" problem, let's discuss more of the details of `scikit-learn` (simply called `sklearn`).

`Scikit-learn` is the main `Python` machine learning library. It consists of many **learners** which can learn models from data, as well as a lot of **utility functions** such as `train_test_split()`.

In `scikit-learn`, an **estimator** is a Python object that implements the methods `fit(X, y)` and `predict(T)`

Let's see the structure of `scikit-learn` needed to make these fits. `fit()` always takes two arguments:
```python
estimator.fit(Xtrain, ytrain)
```
We will consider two estimators in this lab: `LinearRegression` and `KNeighborsRegressor`.

It is very important to understand that `Xtrain` must be in the form of a **2x2 array** with each row corresponding to one sample, and each column corresponding to the feature values for that sample.

`ytrain` on the other hand is a simple array of responses.  These are continuous for regression problems.

![header](figs/featurematrix.png)


---

<a class="anchor" id="sk-practice"></a>
## Practice with `sklearn`: Obtaining training and testing sets from a dataframe

We begin by loading up the `mtcars` dataset. This data was extracted from the 1974 Motor Trend US magazine, and comprises of fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models). We will load this data to a dataframe with 32 observations on 11 (numeric) variables. Here is an explanation of the features:

- `mpg` is Miles/(US) gallon
- `cyl` is Number of cylinders,
- `disp` is	Displacement (cu.in.),
- `hp` is	Gross horsepower,
- `drat` is	Rear axle ratio,
- `wt` is the Weight (1000 lbs),
- `qsec` is 1/4 mile time,
- `vs` is Engine (0 = V-shaped, 1 = straight),
- `am` is Transmission (0 = automatic, 1 = manual),
- `gear` is the Number of forward gears,
- `carb` is	Number of carburetors.

As you will see in **Exercise 5** of Homework 05, the first step you are required to complete to create a regression model using `sklearn` on the `mtcars` dataframe is to: "load the data `mtcars` and split them into a training set and a test set."

While loading the dataframe and analysing its fundamnetal propoerties should be pretty easy for you at this stage, splitting the data into a training and testing set is a new step. Thus, let us complete this first step together.

**NOTE:** you are still invited to repeat the process illustrated in the following code lines in your homework to check whether you understood the underlying concept!

In [ ]:
import pandas as pd

# Loading mtcars dataframe
dfcars = pd.read_csv("data/mtcars.csv")
dfcars.head()

In [ ]:
# Fixing the first column title
dfcars = dfcars.rename(columns={"Unnamed: 0": "car name"})
dfcars.head()

In [ ]:
# Examining the dimensions of the dataframe
dfcars.shape

Next, let's split the dataset into a training set and test set.

In [ ]:
# Splitting into training set and testing set
from sklearn.model_selection import train_test_split

# Setting random_state to get the same split every time
traindf, testdf = train_test_split(dfcars, test_size=0.2, random_state=42)

In [ ]:
# Testing set is around 20% of the total data; training set is around 80%
print("Shape of full dataset is: {0}".format(dfcars.shape))
print("Shape of training dataset is: {0}".format(traindf.shape))
print("Shape of test dataset is: {0}".format(testdf.shape))

**IMPORTANT**
To get to part 6 below, you will have to first complete the homework exercise 5.1 and then use that to complete k-nearest neighbours exercise below. The variables X_train and y_train come from the homework and it is your task to do it. If you are not able to, then solutions will be provided to you.

<a class="anchor" id="kNN"></a>
# $k$-nearest neighbors

Now that you're familiar with `sklearn`, you're ready to do a KNN regression.

Sklearn's regressor is called `sklearn.neighbors.KNeighborsRegressor`. Its main parameter is the **number of nearest neighbors**. There are other parameters such as the distance metric (default for 2 order is the Euclidean distance). For a list of all the parameters see the [Sklearn kNN Regressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

For now, let's use $3$ nearest neighbors.

In [ ]:
# Import the library
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Set number of neighbors
k = 3
knnreg = KNeighborsRegressor(n_neighbors=k)

In [ ]:
# Fit the regressor - make sure your numpy arrays are the right shape!
knnreg.fit(X_train, y_train)

# Evaluate the outcome on the train set using R^2
r2_train = knnreg.score(X_train, y_train)

# Print results
print(f"kNN model with {k} neighbors gives R^2 on the train set: {r2_train}")

We can now analyse the predicted values that our regression model originates from the given test data (X_test).

In [ ]:
knnreg.predict(X_test)

From these estimates our regression model can calculate $R^{2}$ score on the test set.

In [ ]:
r2_test = knnreg.score(X_test, y_test)
print(f"kNN model with {k} neighbors gives R^2 on the test set: {r2_test:.5}")

Not so good? Lets vary the number of neighbors and see what we get.

In [ ]:
# Make our lives easy by storing the different regressors in a dictionary
regdict = {}

# Make our lives easier by entering the k values from a list
k_list = [1, 2, 4, 15]

# Do a bunch of KNN regressions
for k in k_list:
    knnreg = KNeighborsRegressor(n_neighbors=k)
    knnreg.fit(X_train, y_train)
    # Store the regressors in a dictionary
    regdict[k] = knnreg

# Print the dictionary to see what we have
regdict

Now let's plot all the k values in same plot.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(dfcars.wt, dfcars.mpg, "o", label="data")

xgrid = np.linspace(np.min(dfcars.wt), np.max(dfcars.wt), 100)

# let's unpack the dictionary to its elements (items) which is the k and Regressor
for k, regressor in regdict.items():
    predictions = regressor.predict(xgrid.reshape(-1, 1))
    ax.plot(xgrid, predictions, label="{}-NN".format(k))

ax.legend()

**IMPORTANT**
Notice how the $1$-NN goes through every point on the training set but utterly fails elsewhere.

Lets look at the scores on the training set.

In [ ]:
ks = range(1, 15)  # Grid of k's
scores_train = []  # R2 scores
for k in ks:
    # Create KNN model
    knnreg = KNeighborsRegressor(n_neighbors=k)

    # Fit the model to training data
    knnreg.fit(X_train, y_train)

    # Calculate R^2 score
    score_train = knnreg.score(X_train, y_train)
    scores_train.append(score_train)

# Plot
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(ks, scores_train, "o-")
ax.set_xlabel(r"$k$")
ax.set_ylabel(r"$R^{2}$")

<div class="exercise"><b>Exercise</b></div>

* Why do we get a perfect $R^2$ at k=1 for the training set?
* Make the same plot as above on the *test* set.
* What is the best $k$?

Your code here
First try it yourself and if it dosesnt work, try again. Stil doesn't work? Try working with a friend in groups. Nothing? Ok, use the code below.

In [ ]:
# %load solutions/knn_regression.py